In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import logging
import re
from typing import Dict, List, Optional
from datetime import datetime

class EnhancedJobScraper:
    def __init__(self):
        self.base_url = "https://getmatch.ru/vacancies"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        logging.basicConfig(level=logging.INFO)
        self.logger = logging.getLogger(__name__)

    def parse_salary(self, salary_text: str) -> tuple:
        """
        Парсит строку с зарплатой и возвращает минимальное и максимальное значение

        Примеры входных данных:
        - "250 000 — 300 000 ₽/мес на руки"
        - "от 250 000 ₽/мес на руки"
        - "до 300 000 ₽/мес на руки"
        """
        if not salary_text:
            return None, None

        # Удаляем все лишние пробелы и приводим к нижнему регистру
        salary_text = salary_text.lower().strip()

        # Удаляем "₽/мес на руки" и подобные окончания
        salary_text = re.sub(r'₽/мес.*$', '', salary_text)

        # Удаляем все пробелы из чисел
        salary_text = re.sub(r'\s(?=\d)', '', salary_text)

        # Пытаемся найти два числа (диапазон)
        range_match = re.findall(r'\d+', salary_text)

        if 'от' in salary_text and len(range_match) == 1:
            return int(range_match[0]), None
        elif 'до' in salary_text and len(range_match) == 1:
            return None, int(range_match[0])
        elif len(range_match) >= 2:
            return int(range_match[0]), int(range_match[1])
        elif len(range_match) == 1:
            return int(range_match[0]), int(range_match[0])

        return None, None

    def get_job_description(self, job_url: str) -> Dict:
        """Извлекает полную информацию о вакансии"""
        try:
            response = requests.get(job_url, headers=self.headers)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            job_data = {
                'url': job_url,
                'title': None,
                'company_name': None,
                'salary_text': None,
                'salary_from': None,
                'salary_to': None,
                'location': None,
                'work_format': None,
                'specialization': None,
                'level': None,
                'company_logo_url': None,
                'description_text': [],
                'skills': [],
                'posted_date': None
            }

            # Заголовок вакансии
            title_elem = soup.find('h1')
            if title_elem:
                job_data['title'] = title_elem.text.strip()

            # Название компании
            company_elem = soup.find('h2')
            if company_elem and company_elem.find('a'):
                job_data['company_name'] = company_elem.find('a').text.strip()

            # Зарплата
            salary_elem = soup.find('h3')
            if salary_elem:
                salary_text = salary_elem.text.strip()
                job_data['salary_text'] = salary_text
                job_data['salary_from'], job_data['salary_to'] = self.parse_salary(salary_text)

            # Локация и формат работы
            location_container = soup.find('div', class_='b-vacancy-locations')
            if location_container:
                locations = location_container.find_all('span', class_='g-label')
                for loc in locations:
                    if '📍' in loc.text:
                        job_data['location'] = loc.text.replace('📍', '').strip()
                    else:
                        job_data['work_format'] = loc.text.strip()

            # Специализация и уровень
            specs_container = soup.find('div', class_='b-specs')
            if specs_container:
                rows = specs_container.find_all('div', class_='row')
                for row in rows:
                    term = row.find('div', class_='b-term')
                    value = row.find('div', class_='b-value')
                    if term and value:
                        term_text = term.text.strip().lower()
                        if 'специализация' in term_text:
                            job_data['specialization'] = value.text.strip()
                        elif 'уровень' in term_text:
                            job_data['level'] = value.text.strip()

            # Логотип компании
            logo_elem = soup.find('img', {'alt': lambda x: x and 'logo' in x.lower()})
            if logo_elem:
                job_data['company_logo_url'] = logo_elem.get('src')

            # Описание вакансии
            description_sections = [
                ('b-vacancy-short-description', 'Краткое описание'),
                ('b-vacancy-description', 'Полное описание')
            ]

            for class_name, section_name in description_sections:
                section = soup.find('section', class_=class_name)
                if section:
                    for elem in section.stripped_strings:
                        if elem.strip():
                            job_data['description_text'].append(elem.strip())

            # Технологии/навыки
            stack_container = soup.find('div', class_='b-vacancy-stack-container')
            if stack_container:
                skills = [skill.text.strip() for skill in stack_container.find_all('span', class_='g-label')]
                job_data['skills'] = skills

            # Объединяем текст описания
            job_data['description_text'] = '\n'.join(job_data['description_text'])

            return job_data

        except Exception as e:
            self.logger.error(f"Ошибка при получении описания вакансии {job_url}: {e}")
            return {'url': job_url, 'error': str(e)}

    def get_job_urls(self, page: int = 1) -> List[str]:
        """Получение списка URL вакансий с страницы"""
        params = {"p": page}
        try:
            response = requests.get(self.base_url, params=params, headers=self.headers)
            response.raise_for_status()

            soup = BeautifulSoup(response.text, 'html.parser')
            job_cards = soup.find_all('div', class_='b-vacancy-card')

            job_urls = []
            for card in job_cards:
                title_elem = card.find('h3')
                if title_elem:
                    link_elem = title_elem.find('a')
                    if link_elem and link_elem.get('href'):
                        job_urls.append('https://getmatch.ru' + link_elem.get('href'))

            return job_urls
        except Exception as e:
            self.logger.error(f"Ошибка при получении списка вакансий со страницы {page}: {e}")
            return []

    def scrape_to_csv(self, num_pages: int = 5, output_format: str = 'csv'):
        """Сбор описаний вакансий с нескольких страниц"""
        all_jobs = []

        for page in range(1, num_pages + 1):
            self.logger.info(f"Обработка страницы {page}...")
            job_urls = self.get_job_urls(page)

            for url in job_urls:
                self.logger.info(f"Получение описания вакансии: {url}")
                job_data = self.get_job_description(url)
                if job_data and 'error' not in job_data:
                    all_jobs.append(job_data)
                time.sleep(1)  # Задержка между запросами

            time.sleep(2)  # Задержка между страницами

        if all_jobs:
            timestamp = time.strftime("%Y%m%d_%H%M%S")

            if output_format == 'csv':
                df = pd.DataFrame(all_jobs)
                filename = f'job_descriptions_{timestamp}.csv'
                df.to_csv(filename, index=False, encoding='utf-8')
                self.logger.info(f"Сохранено {len(all_jobs)} описаний вакансий в файл {filename}")
            elif output_format == 'json':
                filename = f'job_descriptions_{timestamp}.json'
                pd.DataFrame(all_jobs).to_json(filename, orient='records', force_ascii=False, indent=2)
                self.logger.info(f"Сохранено {len(all_jobs)} описаний вакансий в файл {filename}")
        else:
            self.logger.error("Не удалось собрать данные о вакансиях")

if __name__ == "__main__":
    scraper = EnhancedJobScraper()
    scraper.scrape_to_csv(num_pages=69)